In [3]:
from src.dataset import Darknet
from src.models import ClassificationModel
from src.utils.functions import mkdir_if_not_exists, verify_existence_data
import tensorflow as tf
from tqdm import tqdm

In [4]:
data_path = f'dataset/Darknet/'
darknet_dataset = Darknet(data_path)
if not verify_existence_data(f"{data_path}/processed/darknet_dataset_processed_encoded.csv") and not verify_existence_data(f"{data_path}/processed/darknet_dataset_processed.csv"):
    darknet_dataset.exportProcessedData()
darknet_data, darknet_data_models = darknet_dataset.getProcessedData()

In [5]:
train, test, val = darknet_dataset.getTrainTestValData(0.33, True)

In [6]:
train_ds = darknet_dataset.convertToDataset(train, batch_size=256)
test_ds = darknet_dataset.convertToDataset(test, shuffle=False, batch_size=256)
val_ds = darknet_dataset.convertToDataset(val, shuffle=False, batch_size=256)

In [7]:
data_columns = darknet_data_models.columns
integer_category_columns = data_columns[:108]
real_columns = data_columns[108:len(data_columns) - 2]

inputs = []
encoded_features =[]

In [8]:
print("[!] - Normalizing real columns", end="\n")
for i in tqdm(range(len(real_columns))):
    numeric_col = tf.keras.Input(shape=(1, ), name=real_columns[i])
    normalization_layer = darknet_dataset.getNormalizationLayer(real_columns[i], train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

[!] - Normalizating real columns


100%|██████████| 76/76 [16:52<00:00, 13.32s/it]


In [9]:
print("[!] - Encoding integer and string layers", end="\n")
for i in tqdm(range(len(integer_category_columns))):
    categorical_col = tf.keras.Input(shape=(1, ), name=integer_category_columns[i], dtype='int64')
    encoding_layer = darknet_dataset.getCategoryEncodingLayer(integer_category_columns[i], train_ds, dtype='int64', max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

[!] - Encoding integer and string layers


100%|██████████| 108/108 [31:20<00:00, 17.41s/it]


In [10]:
all_features = tf.keras.layers.concatenate(encoded_features)

In [14]:
x = tf.keras.layers.Dense(32, activation='relu')(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, output)
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
col_0 (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
col_1 (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
col_2 (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
col_3 (InputLayer)              [(None, 1)]          0                                            
____________________________________________________________________________________________

In [15]:
fit_model = model.fit(train_ds, epochs=20, validation_data = val_ds)
print(fit_model.history.keys())

Epoch 1/20
249/249 [==============================] - 46s 105ms/step - loss: 0.1376 - accuracy: 0.9523 - val_loss: 0.0245 - val_accuracy: 0.9947
Epoch 2/20
249/249 [==============================] - 32s 86ms/step - loss: 0.0300 - accuracy: 0.9935 - val_loss: 0.0143 - val_accuracy: 0.9964
Epoch 3/20
249/249 [==============================] - 32s 83ms/step - loss: 0.0203 - accuracy: 0.9955 - val_loss: 0.0109 - val_accuracy: 0.9975
Epoch 4/20
249/249 [==============================] - 33s 86ms/step - loss: 0.0153 - accuracy: 0.9964 - val_loss: 0.0095 - val_accuracy: 0.9979
Epoch 5/20
249/249 [==============================] - 31s 84ms/step - loss: 0.0130 - accuracy: 0.9970 - val_loss: 0.0087 - val_accuracy: 0.9978
Epoch 6/20
249/249 [==============================] - 33s 85ms/step - loss: 0.0112 - accuracy: 0.9975 - val_loss: 0.0079 - val_accuracy: 0.9981
Epoch 7/20
249/249 [==============================] - 34s 88ms/step - loss: 0.0105 - accuracy: 0.9975 - val_loss: 0.0080 - val_accuracy

In [16]:
loss, acc = model.evaluate(test_ds)
print(loss, acc)

183/183 [==============================] - 10s 56ms/step - loss: 0.0057 - accuracy: 0.9986
0.005709651857614517 0.9986292123794556


In [17]:
print(model.metrics_names)

['loss', 'accuracy']


In [25]:
model.save('etc/trainModel/orgClassifier')

INFO:tensorflow:Assets written to: etc/trainModel/orgClassifier\assets


INFO:tensorflow:Assets written to: etc/trainModel/orgClassifier\assets
